In [1]:
import os
import numpy as np
import pandas as pd
from ete3 import Tree
import random
import subprocess
random.seed(191919)  # Set Random Seed for Reproducibility

In [2]:
##### USER INPUT #################################

### INPUT/OUTPUT FOLDER ###
outPath = '/Users/marcellabaiz/Documents/Toews_Lab/manuscripts/parulidGM/reviews:revision/treecmp/results_braycurtis/'

##################################################

### PATH TO THE HOST TREE (phylogeny)###
htPath = '/Users/marcellabaiz/Documents/Toews_Lab/16s_data/Jan2020/TreeCmp/newick_warblers.newick'

### PATH TO diet DENDROGRAM (OR ALTERNATIVE TREE) ###
mtPath = '/Users/marcellabaiz/Documents/Toews_Lab/warbler_diet/grouped_diet/topology_analyses/subset_all1/newick_bray_grouped.newick'

### NUMBER OF RANDOM TREES ###
numRandom = 10000

In [3]:
##### IMPORT AND CLEAN TREES #########################################################

### IMPORT HOST TREE ###
htIn = Tree(htPath)
### IMPORT MICROBIOME TREE ###
mtIn = Tree(mtPath)

### PRINT THE TREES ###
print('\n --- HOST TREE --- ')
print(htIn)
print('\n --- MICROBIOME TREE --- ')
print(mtIn)

### LIST TO STORE HOST LEAVES ###
htLeafs = []

### CHECK THAT TREES HAVE SAME NUMBER OF LEAVES ###
print('\n --- CHECKING TREES --- \n')
if len(htIn) == len(mtIn):   
    
    ### FOR EACH LEAF IN THE HOST TREE MAKE SURE ALSO IN MICROBIOME TREE ###
    for leaf in mtIn:
        
        ### STORE LEAF NAME ###
        htLeafs.append(leaf.name)
        
        ### CHECK MICROBIOME TREE ###
        if leaf.name not in mtIn:
            print('ERROR - '+leaf.name+' Not Found in Microbiome Tree!')

### # IF NOT SAME NUMBER OF LEAVES PRINT ERROR ###
else: print('ERROR - Host and Microbiome Tree Have Different Numbers of Nodes!')

##### GENERATING RANDOM TREES ########################################################

### PATH TO STORE RANDOM TREES ###
rtPath = outPath+'/random_trees/'

### CREATE OUTPUT FOLDER ###
if not os.path.isdir(rtPath): os.makedirs(rtPath)

### FOR EACH RANDOM TREE ###
for curRandom in np.arange(numRandom):
    
    ### INTITALIZE RANDOM TREE ###
    t = Tree()
    
    ### SHUFFLE LEAVES ###
    random.shuffle(htLeafs)
    
    ### POPULATE TREE ###
    t.populate(len(htIn), names_library=htLeafs)
    
    ### WRITE TREE ###
    t.write(outfile=rtPath+'/tree_'+str(curRandom)+'.newick')

### CONCATENATE THE RANDOM TREES INTO SINGLE FILE ###
subprocess.check_output(str('for i in '+outPath+'/random_trees/*; do  cat $i >> '+outPath+'/random_trees.newick; done'), shell=True)

##### COMARE WITH TREECMP METHODS ####################################################

### FOR EACH TREECMP METHOD ###
methods = {"rc":'Rooted Robinson-Foulds', "mc":'Rooted Matching Cluster', "ms":'Unrooted Matching Split', "rf":'Unrooted Robinson-Foulds'}

for method in ["rc", "mc", "rf", "ms"]:
    
    ### MAKE COMPARISON FOLDER ###
    if not os.path.isdir(outPath+'/compare_'+method+'/'): os.makedirs(outPath+'/compare_'+method+'/')
        
    ### COMPARE HOST AND MICROBIOME TREE ###
    subprocess.check_output(str('java -jar /Users/marcellabaiz/Documents/Toews_Lab/16s_data/Jan2020/TreeCmp/TreeCmp_v2.0-b76/bin/treeCmp.jar -r '+htPath+
              ' -d '+method+' -i '+mtPath+' -o '+outPath+'/compare_'+method+'/compare_ht_mt.txt -N'), shell=True)
    
    ### COMPARE HOST AND RANDOM TREES ###
    subprocess.check_output(str('java -jar /Users/marcellabaiz/Documents/Toews_Lab/16s_data/Jan2020/TreeCmp/TreeCmp_v2.0-b76/bin/treeCmp.jar -r '+htPath+
              ' -d '+method+' -i '+outPath+'/random_trees.newick -o '+outPath+'/compare_'+method+'/compare_ht_random.txt -N'), shell=True)

    ### READ IN COMPARISON RESULTS ###
    mtCompare = pd.read_csv(outPath+'/compare_'+method+'/compare_ht_mt.txt', sep='\t')
    rtCompare = pd.read_csv(outPath+'/compare_'+method+'/compare_ht_random.txt', sep='\t')

    ### PRINT RESULTS ###
    print(' --- METHOD: '+methods[method]+' --- ')
    print("Host-Microbe Score:   " + str(mtCompare.iloc[0,3]))
    print("Max Stochastic Metric:  " + str(max(rtCompare.iloc[:,3])))
    print('Normalized Score: '+str(mtCompare.iloc[0,3]/max(rtCompare.iloc[:,3])))
    print('Random Trees with Equivalent or More Congruent Score: '+str(len(rtCompare[rtCompare.iloc[:,3] <= mtCompare.iloc[0,3]])))
    print('Total Trees: '+ str(len(rtCompare)))
    print('P-Value: '+str(len(rtCompare[rtCompare.iloc[:,3] <= mtCompare.iloc[0,3]]) / len(rtCompare))+'\n')


 --- HOST TREE --- 

   /-OVEN
  |
  |   /-BAWW
--|  |
  |  |      /-NAWA
  |  |   /-|
  |  |  |   \-COYE
   \-|  |
     |  |         /-HOWA
     |  |      /-|
     |  |     |   \-AMRE
     |  |     |
      \-|   /-|   /-BTBW
        |  |  |  |
        |  |  |  |         /-NOPA
        |  |  |  |      /-|
        |  |   \-|     |   \-MAWA
        |  |     |   /-|
        |  |     |  |  |   /-BLBW
         \-|     |  |   \-|
           |      \-|      \-CSWA
           |        |
           |        |   /-MYWA
           |         \-|
           |            \-BTNW
           |
            \-CAWA

 --- MICROBIOME TREE --- 

      /-BTBW
   /-|
  |   \-HOWA
  |
--|      /-COYE
  |   /-|
  |  |  |   /-CAWA
  |  |   \-|
   \-|      \-OVEN
     |
     |   /-AMRE
     |  |
      \-|   /-BAWW
        |  |
         \-|   /-NOPA
           |  |
           |  |      /-CSWA
            \-|   /-|
              |  |  |   /-BLBW
              |  |   \-|
               \-|      \-MAWA
              

In [4]:
##### USER INPUT #################################

### INPUT/OUTPUT FOLDER ###
outPath = '/Users/marcellabaiz/Documents/Toews_Lab/manuscripts/parulidGM/reviews:revision/treecmp/results_jaccard/'

##################################################

### PATH TO THE HOST TREE (phylogeny)###
htPath = '/Users/marcellabaiz/Documents/Toews_Lab/16s_data/Jan2020/TreeCmp/newick_warblers.newick'

### PATH TO diet DENDROGRAM (OR ALTERNATIVE TREE) ###
mtPath = '/Users/marcellabaiz/Documents/Toews_Lab/warbler_diet/grouped_diet/topology_analyses/subset_all1/newick_jaccard_grouped.newick'

### NUMBER OF RANDOM TREES ###
numRandom = 10000

In [5]:
##### IMPORT AND CLEAN TREES #########################################################

### IMPORT HOST TREE ###
htIn = Tree(htPath)
### IMPORT MICROBIOME TREE ###
mtIn = Tree(mtPath)

### PRINT THE TREES ###
print('\n --- HOST TREE --- ')
print(htIn)
print('\n --- MICROBIOME TREE --- ')
print(mtIn)

### LIST TO STORE HOST LEAVES ###
htLeafs = []

### CHECK THAT TREES HAVE SAME NUMBER OF LEAVES ###
print('\n --- CHECKING TREES --- \n')
if len(htIn) == len(mtIn):   
    
    ### FOR EACH LEAF IN THE HOST TREE MAKE SURE ALSO IN MICROBIOME TREE ###
    for leaf in mtIn:
        
        ### STORE LEAF NAME ###
        htLeafs.append(leaf.name)
        
        ### CHECK MICROBIOME TREE ###
        if leaf.name not in mtIn:
            print('ERROR - '+leaf.name+' Not Found in Microbiome Tree!')

### # IF NOT SAME NUMBER OF LEAVES PRINT ERROR ###
else: print('ERROR - Host and Microbiome Tree Have Different Numbers of Nodes!')

##### GENERATING RANDOM TREES ########################################################

### PATH TO STORE RANDOM TREES ###
rtPath = outPath+'/random_trees/'

### CREATE OUTPUT FOLDER ###
if not os.path.isdir(rtPath): os.makedirs(rtPath)

### FOR EACH RANDOM TREE ###
for curRandom in np.arange(numRandom):
    
    ### INTITALIZE RANDOM TREE ###
    t = Tree()
    
    ### SHUFFLE LEAVES ###
    random.shuffle(htLeafs)
    
    ### POPULATE TREE ###
    t.populate(len(htIn), names_library=htLeafs)
    
    ### WRITE TREE ###
    t.write(outfile=rtPath+'/tree_'+str(curRandom)+'.newick')

### CONCATENATE THE RANDOM TREES INTO SINGLE FILE ###
subprocess.check_output(str('for i in '+outPath+'/random_trees/*; do  cat $i >> '+outPath+'/random_trees.newick; done'), shell=True)

##### COMARE WITH TREECMP METHODS ####################################################

### FOR EACH TREECMP METHOD ###
methods = {"rc":'Rooted Robinson-Foulds', "mc":'Rooted Matching Cluster', "ms":'Unrooted Matching Split', "rf":'Unrooted Robinson-Foulds'}

for method in ["rc", "mc", "rf", "ms"]:
    
    ### MAKE COMPARISON FOLDER ###
    if not os.path.isdir(outPath+'/compare_'+method+'/'): os.makedirs(outPath+'/compare_'+method+'/')
        
    ### COMPARE HOST AND MICROBIOME TREE ###
    subprocess.check_output(str('java -jar /Users/marcellabaiz/Documents/Toews_Lab/16s_data/Jan2020/TreeCmp/TreeCmp_v2.0-b76/bin/treeCmp.jar -r '+htPath+
              ' -d '+method+' -i '+mtPath+' -o '+outPath+'/compare_'+method+'/compare_ht_mt.txt -N'), shell=True)
    
    ### COMPARE HOST AND RANDOM TREES ###
    subprocess.check_output(str('java -jar /Users/marcellabaiz/Documents/Toews_Lab/16s_data/Jan2020/TreeCmp/TreeCmp_v2.0-b76/bin/treeCmp.jar -r '+htPath+
              ' -d '+method+' -i '+outPath+'/random_trees.newick -o '+outPath+'/compare_'+method+'/compare_ht_random.txt -N'), shell=True)

    ### READ IN COMPARISON RESULTS ###
    mtCompare = pd.read_csv(outPath+'/compare_'+method+'/compare_ht_mt.txt', sep='\t')
    rtCompare = pd.read_csv(outPath+'/compare_'+method+'/compare_ht_random.txt', sep='\t')

    ### PRINT RESULTS ###
    print(' --- METHOD: '+methods[method]+' --- ')
    print("Host-Microbe Score:   " + str(mtCompare.iloc[0,3]))
    print("Max Stochastic Metric:  " + str(max(rtCompare.iloc[:,3])))
    print('Normalized Score: '+str(mtCompare.iloc[0,3]/max(rtCompare.iloc[:,3])))
    print('Random Trees with Equivalent or More Congruent Score: '+str(len(rtCompare[rtCompare.iloc[:,3] <= mtCompare.iloc[0,3]])))
    print('Total Trees: '+ str(len(rtCompare)))
    print('P-Value: '+str(len(rtCompare[rtCompare.iloc[:,3] <= mtCompare.iloc[0,3]]) / len(rtCompare))+'\n')


 --- HOST TREE --- 

   /-OVEN
  |
  |   /-BAWW
--|  |
  |  |      /-NAWA
  |  |   /-|
  |  |  |   \-COYE
   \-|  |
     |  |         /-HOWA
     |  |      /-|
     |  |     |   \-AMRE
     |  |     |
      \-|   /-|   /-BTBW
        |  |  |  |
        |  |  |  |         /-NOPA
        |  |  |  |      /-|
        |  |   \-|     |   \-MAWA
        |  |     |   /-|
        |  |     |  |  |   /-BLBW
         \-|     |  |   \-|
           |      \-|      \-CSWA
           |        |
           |        |   /-MYWA
           |         \-|
           |            \-BTNW
           |
            \-CAWA

 --- MICROBIOME TREE --- 

   /-HOWA
  |
--|   /-AMRE
  |  |
   \-|   /-COYE
     |  |
      \-|   /-BAWW
        |  |
        |  |   /-OVEN
         \-|  |
           |  |      /-BTBW
           |  |     |
           |  |   /-|      /-CSWA
            \-|  |  |   /-|
              |  |  |  |   \-CAWA
              |  |   \-|
              |  |     |   /-NOPA
              |  |      \-|
     

In [6]:
##### USER INPUT #################################

### INPUT/OUTPUT FOLDER ###
outPath = '/Users/marcellabaiz/Documents/Toews_Lab/manuscripts/parulidGM/reviews:revision/treecmp/results_unifrac/'

##################################################

### PATH TO THE HOST TREE (phylogeny)###
htPath = '/Users/marcellabaiz/Documents/Toews_Lab/16s_data/Jan2020/TreeCmp/newick_warblers.newick'

### PATH TO diet DENDROGRAM (OR ALTERNATIVE TREE) ###
mtPath = '/Users/marcellabaiz/Documents/Toews_Lab/warbler_diet/grouped_diet/topology_analyses/subset_all1/newick_unifrac_grouped.newick'

### NUMBER OF RANDOM TREES ###
numRandom = 10000

In [7]:
##### IMPORT AND CLEAN TREES #########################################################

### IMPORT HOST TREE ###
htIn = Tree(htPath)
### IMPORT MICROBIOME TREE ###
mtIn = Tree(mtPath)

### PRINT THE TREES ###
print('\n --- HOST TREE --- ')
print(htIn)
print('\n --- MICROBIOME TREE --- ')
print(mtIn)

### LIST TO STORE HOST LEAVES ###
htLeafs = []

### CHECK THAT TREES HAVE SAME NUMBER OF LEAVES ###
print('\n --- CHECKING TREES --- \n')
if len(htIn) == len(mtIn):   
    
    ### FOR EACH LEAF IN THE HOST TREE MAKE SURE ALSO IN MICROBIOME TREE ###
    for leaf in mtIn:
        
        ### STORE LEAF NAME ###
        htLeafs.append(leaf.name)
        
        ### CHECK MICROBIOME TREE ###
        if leaf.name not in mtIn:
            print('ERROR - '+leaf.name+' Not Found in Microbiome Tree!')

### # IF NOT SAME NUMBER OF LEAVES PRINT ERROR ###
else: print('ERROR - Host and Microbiome Tree Have Different Numbers of Nodes!')

##### GENERATING RANDOM TREES ########################################################

### PATH TO STORE RANDOM TREES ###
rtPath = outPath+'/random_trees/'

### CREATE OUTPUT FOLDER ###
if not os.path.isdir(rtPath): os.makedirs(rtPath)

### FOR EACH RANDOM TREE ###
for curRandom in np.arange(numRandom):
    
    ### INTITALIZE RANDOM TREE ###
    t = Tree()
    
    ### SHUFFLE LEAVES ###
    random.shuffle(htLeafs)
    
    ### POPULATE TREE ###
    t.populate(len(htIn), names_library=htLeafs)
    
    ### WRITE TREE ###
    t.write(outfile=rtPath+'/tree_'+str(curRandom)+'.newick')

### CONCATENATE THE RANDOM TREES INTO SINGLE FILE ###
subprocess.check_output(str('for i in '+outPath+'/random_trees/*; do  cat $i >> '+outPath+'/random_trees.newick; done'), shell=True)

##### COMARE WITH TREECMP METHODS ####################################################

### FOR EACH TREECMP METHOD ###
methods = {"rc":'Rooted Robinson-Foulds', "mc":'Rooted Matching Cluster', "ms":'Unrooted Matching Split', "rf":'Unrooted Robinson-Foulds'}

for method in ["rc", "mc", "rf", "ms"]:
    
    ### MAKE COMPARISON FOLDER ###
    if not os.path.isdir(outPath+'/compare_'+method+'/'): os.makedirs(outPath+'/compare_'+method+'/')
        
    ### COMPARE HOST AND MICROBIOME TREE ###
    subprocess.check_output(str('java -jar /Users/marcellabaiz/Documents/Toews_Lab/16s_data/Jan2020/TreeCmp/TreeCmp_v2.0-b76/bin/treeCmp.jar -r '+htPath+
              ' -d '+method+' -i '+mtPath+' -o '+outPath+'/compare_'+method+'/compare_ht_mt.txt -N'), shell=True)
    
    ### COMPARE HOST AND RANDOM TREES ###
    subprocess.check_output(str('java -jar /Users/marcellabaiz/Documents/Toews_Lab/16s_data/Jan2020/TreeCmp/TreeCmp_v2.0-b76/bin/treeCmp.jar -r '+htPath+
              ' -d '+method+' -i '+outPath+'/random_trees.newick -o '+outPath+'/compare_'+method+'/compare_ht_random.txt -N'), shell=True)

    ### READ IN COMPARISON RESULTS ###
    mtCompare = pd.read_csv(outPath+'/compare_'+method+'/compare_ht_mt.txt', sep='\t')
    rtCompare = pd.read_csv(outPath+'/compare_'+method+'/compare_ht_random.txt', sep='\t')

    ### PRINT RESULTS ###
    print(' --- METHOD: '+methods[method]+' --- ')
    print("Host-Microbe Score:   " + str(mtCompare.iloc[0,3]))
    print("Max Stochastic Metric:  " + str(max(rtCompare.iloc[:,3])))
    print('Normalized Score: '+str(mtCompare.iloc[0,3]/max(rtCompare.iloc[:,3])))
    print('Random Trees with Equivalent or More Congruent Score: '+str(len(rtCompare[rtCompare.iloc[:,3] <= mtCompare.iloc[0,3]])))
    print('Total Trees: '+ str(len(rtCompare)))
    print('P-Value: '+str(len(rtCompare[rtCompare.iloc[:,3] <= mtCompare.iloc[0,3]]) / len(rtCompare))+'\n')


 --- HOST TREE --- 

   /-OVEN
  |
  |   /-BAWW
--|  |
  |  |      /-NAWA
  |  |   /-|
  |  |  |   \-COYE
   \-|  |
     |  |         /-HOWA
     |  |      /-|
     |  |     |   \-AMRE
     |  |     |
      \-|   /-|   /-BTBW
        |  |  |  |
        |  |  |  |         /-NOPA
        |  |  |  |      /-|
        |  |   \-|     |   \-MAWA
        |  |     |   /-|
        |  |     |  |  |   /-BLBW
         \-|     |  |   \-|
           |      \-|      \-CSWA
           |        |
           |        |   /-MYWA
           |         \-|
           |            \-BTNW
           |
            \-CAWA

 --- MICROBIOME TREE --- 

   /-BAWW
  |
  |      /-OVEN
  |   /-|
  |  |  |   /-BTBW
--|  |   \-|
  |  |      \-HOWA
  |  |
  |  |         /-BTNW
  |  |        |
  |  |      /-|   /-BLBW
   \-|     |  |  |
     |     |   \-|      /-NOPA
     |     |     |   /-|
     |     |     |  |   \-MYWA
     |   /-|      \-|
     |  |  |        |   /-CSWA
     |  |  |         \-|
     |  |  |           

In [8]:
##### USER INPUT #################################

### INPUT/OUTPUT FOLDER ###
outPath = '/Users/marcellabaiz/Documents/Toews_Lab/manuscripts/parulidGM/reviews:revision/treecmp/results_weightedunifrac/'

##################################################

### PATH TO THE HOST TREE (phylogeny)###
htPath = '/Users/marcellabaiz/Documents/Toews_Lab/16s_data/Jan2020/TreeCmp/newick_warblers.newick'

### PATH TO diet DENDROGRAM (OR ALTERNATIVE TREE) ###
mtPath = '/Users/marcellabaiz/Documents/Toews_Lab/warbler_diet/grouped_diet/topology_analyses/subset_all1/newick_weightedunifrac_grouped.newick'

### NUMBER OF RANDOM TREES ###
numRandom = 10000

In [9]:
##### IMPORT AND CLEAN TREES #########################################################

### IMPORT HOST TREE ###
htIn = Tree(htPath)
### IMPORT MICROBIOME TREE ###
mtIn = Tree(mtPath)

### PRINT THE TREES ###
print('\n --- HOST TREE --- ')
print(htIn)
print('\n --- MICROBIOME TREE --- ')
print(mtIn)

### LIST TO STORE HOST LEAVES ###
htLeafs = []

### CHECK THAT TREES HAVE SAME NUMBER OF LEAVES ###
print('\n --- CHECKING TREES --- \n')
if len(htIn) == len(mtIn):   
    
    ### FOR EACH LEAF IN THE HOST TREE MAKE SURE ALSO IN MICROBIOME TREE ###
    for leaf in mtIn:
        
        ### STORE LEAF NAME ###
        htLeafs.append(leaf.name)
        
        ### CHECK MICROBIOME TREE ###
        if leaf.name not in mtIn:
            print('ERROR - '+leaf.name+' Not Found in Microbiome Tree!')

### # IF NOT SAME NUMBER OF LEAVES PRINT ERROR ###
else: print('ERROR - Host and Microbiome Tree Have Different Numbers of Nodes!')

##### GENERATING RANDOM TREES ########################################################

### PATH TO STORE RANDOM TREES ###
rtPath = outPath+'/random_trees/'

### CREATE OUTPUT FOLDER ###
if not os.path.isdir(rtPath): os.makedirs(rtPath)

### FOR EACH RANDOM TREE ###
for curRandom in np.arange(numRandom):
    
    ### INTITALIZE RANDOM TREE ###
    t = Tree()
    
    ### SHUFFLE LEAVES ###
    random.shuffle(htLeafs)
    
    ### POPULATE TREE ###
    t.populate(len(htIn), names_library=htLeafs)
    
    ### WRITE TREE ###
    t.write(outfile=rtPath+'/tree_'+str(curRandom)+'.newick')

### CONCATENATE THE RANDOM TREES INTO SINGLE FILE ###
subprocess.check_output(str('for i in '+outPath+'/random_trees/*; do  cat $i >> '+outPath+'/random_trees.newick; done'), shell=True)

##### COMARE WITH TREECMP METHODS ####################################################

### FOR EACH TREECMP METHOD ###
methods = {"rc":'Rooted Robinson-Foulds', "mc":'Rooted Matching Cluster', "ms":'Unrooted Matching Split', "rf":'Unrooted Robinson-Foulds'}

for method in ["rc", "mc", "rf", "ms"]:
    
    ### MAKE COMPARISON FOLDER ###
    if not os.path.isdir(outPath+'/compare_'+method+'/'): os.makedirs(outPath+'/compare_'+method+'/')
        
    ### COMPARE HOST AND MICROBIOME TREE ###
    subprocess.check_output(str('java -jar /Users/marcellabaiz/Documents/Toews_Lab/16s_data/Jan2020/TreeCmp/TreeCmp_v2.0-b76/bin/treeCmp.jar -r '+htPath+
              ' -d '+method+' -i '+mtPath+' -o '+outPath+'/compare_'+method+'/compare_ht_mt.txt -N'), shell=True)
    
    ### COMPARE HOST AND RANDOM TREES ###
    subprocess.check_output(str('java -jar /Users/marcellabaiz/Documents/Toews_Lab/16s_data/Jan2020/TreeCmp/TreeCmp_v2.0-b76/bin/treeCmp.jar -r '+htPath+
              ' -d '+method+' -i '+outPath+'/random_trees.newick -o '+outPath+'/compare_'+method+'/compare_ht_random.txt -N'), shell=True)

    ### READ IN COMPARISON RESULTS ###
    mtCompare = pd.read_csv(outPath+'/compare_'+method+'/compare_ht_mt.txt', sep='\t')
    rtCompare = pd.read_csv(outPath+'/compare_'+method+'/compare_ht_random.txt', sep='\t')

    ### PRINT RESULTS ###
    print(' --- METHOD: '+methods[method]+' --- ')
    print("Host-Microbe Score:   " + str(mtCompare.iloc[0,3]))
    print("Max Stochastic Metric:  " + str(max(rtCompare.iloc[:,3])))
    print('Normalized Score: '+str(mtCompare.iloc[0,3]/max(rtCompare.iloc[:,3])))
    print('Random Trees with Equivalent or More Congruent Score: '+str(len(rtCompare[rtCompare.iloc[:,3] <= mtCompare.iloc[0,3]])))
    print('Total Trees: '+ str(len(rtCompare)))
    print('P-Value: '+str(len(rtCompare[rtCompare.iloc[:,3] <= mtCompare.iloc[0,3]]) / len(rtCompare))+'\n')


 --- HOST TREE --- 

   /-OVEN
  |
  |   /-BAWW
--|  |
  |  |      /-NAWA
  |  |   /-|
  |  |  |   \-COYE
   \-|  |
     |  |         /-HOWA
     |  |      /-|
     |  |     |   \-AMRE
     |  |     |
      \-|   /-|   /-BTBW
        |  |  |  |
        |  |  |  |         /-NOPA
        |  |  |  |      /-|
        |  |   \-|     |   \-MAWA
        |  |     |   /-|
        |  |     |  |  |   /-BLBW
         \-|     |  |   \-|
           |      \-|      \-CSWA
           |        |
           |        |   /-MYWA
           |         \-|
           |            \-BTNW
           |
            \-CAWA

 --- MICROBIOME TREE --- 

   /-NOPA
  |
  |      /-COYE
--|   /-|
  |  |  |   /-CAWA
  |  |   \-|
   \-|      \-OVEN
     |
     |   /-AMRE
     |  |
      \-|   /-BTNW
        |  |
        |  |      /-NAWA
         \-|   /-|
           |  |   \-MYWA
           |  |
            \-|      /-CSWA
              |   /-|
              |  |  |   /-BLBW
              |  |   \-|
               \-|   